# Digital Cockpit demo


### **Environmental Recognition Agent**
---
- **Role:** This agent is responsible for understanding the environment.
- **Functions:** 
  - Recognizes user behavior and the surrounding context.
  - Creates a "Context" to inform subsequent agents about the current environment.
- **Components:**
  - Speech recognition module.
  - Environmental recognition module.
  - Environmental sensor processing module.
- **Demo Architecture:**
  - Prepare the sample dataset
  - The agent consists of a singble SLM(Phi-3.5 vision). Use multimodal version to support image input.
  - Uge Phi-3.5 vision model with Serverless API that is provided by [Azure AI Model Catalog](https://learn.microsoft.com/en-us/azure/machine-learning/concept-model-catalog?view=azureml-api-2).

In [3]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

In [4]:
# Load environment variables
load_dotenv(override=True)

True

In [5]:
env_agent_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT_PHI35_VISION"],
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL_PHI35_VISION"]),
)

In [6]:
# Check the status of the model
env_agent_model_info = env_agent_client.get_model_info()
print("Model name:", env_agent_model_info.model_name)
print("Model type:", env_agent_model_info.model_type)
print("Model provider name:", env_agent_model_info.model_provider_name)

Model name: phi35-vision-instruct
Model type: chat-completion
Model provider name: Phi


In [7]:
# Data to be sent to the model

# Vehicle data
VehicleData = {
    "InteriorTemperature": 25.5,  # Interior temperature (°C)
    "ExteriorTemperature": 30.2,  # Exterior temperature (°C)
    "Humidity": 60,  # Humidity (%)
    "Weather": "Sunny",  # Weather condition
    "Speed": "80km/h",  # Vehicle speed
    "Acceleration": "1.2m/s²",  # Acceleration
    "SteeringAngle": "15°",  # Steering wheel angle
    "FuelLevel": 45,  # Fuel level (%)
    "EngineRPM": 2500,  # Engine revolutions per minute (RPM)
    "BatteryVoltage": 12.8  # Battery voltage (V)
}

# Driver state
DriverState = {
    "HeartRate": 72,  # Driver's heart rate (BPM)
    "EyeStatus": "Open",  # Eye status
    "Yawning": False,  # Yawning detection
    "AttentionLevel": "High"  # Attention level
}

# Window status
WindowStatus = {
    "FrontLeft": "Closed",  # Front left window
    "FrontRight": "Closed",  # Front right window
    "RearLeft": "Open",  # Rear left window
    "RearRight": "Closed"  # Rear right window
}

# Door status
DoorStatus = {
    "FrontLeft": "Closed",  # Front left door
    "FrontRight": "Closed",  # Front right door
    "RearLeft": "Closed",  # Rear left door
    "RearRight": "Closed",  # Rear right door
    "Trunk": "Closed"  # Trunk
}

# CAN data
CANData = {
    "BrakePressure": 2.3,  # Brake pressure (bar)
    "ThrottlePosition": 25.4,  # Throttle position (%)
    "GearPosition": "Drive",  # Gear position
    "TurnSignal": "Off"  # Turn signal status
}

In [8]:
# Prompt definition
env_agent_system_message = f"""
You are an Environmental Recognition Agent.
Based on the data collected from the car's sensors, generate JSON-format data that represents the current state of the vehicle and driver.
"""

env_agent_user_message = f"""
# Input
"VehicleData": {VehicleData},
"DriverState": {DriverState},
"WindowStatus": {WindowStatus},
"DoorStatus": {DoorStatus},
"CANData": {CANData}
"""

In [9]:
from azure.ai.inference.models import SystemMessage, UserMessage

env_agent_response = env_agent_client.complete(
    messages=[
        SystemMessage(content=env_agent_system_message),
        UserMessage(content=env_agent_user_message),
    ],
)

In [10]:
print("Response:", env_agent_response.choices[0].message.content)
print("Model:", env_agent_response.model)
print("Usage:")
print("\tPrompt tokens:", env_agent_response.usage.prompt_tokens)
print("\tTotal tokens:", env_agent_response.usage.total_tokens)
print("\tCompletion tokens:", env_agent_response.usage.completion_tokens)

Response:  {
  "VehicleData": {
    "InteriorTemperature": 25.5,
    "ExteriorTemperature": 30.2,
    "Humidity": 60,
    "Weather": "Sunny",
    "Speed": 80,
    "Acceleration": 1.2,
    "SteeringAngle": 15,
    "FuelLevel": 45,
    "EngineRPM": 2500,
    "BatteryVoltage": 12.8
  },
  "DriverState": {
    "HeartRate": 72,
    "EyeStatus": "Open",
    "Yawning": false,
    "AttentionLevel": "High"
  },
  "WindowStatus": {
    "FrontLeft": "Closed",
    "FrontRight": "Closed",
    "RearLeft": "Open",
    "RearRight": "Closed"
  },
  "DoorStatus": {
    "FrontLeft": "Closed",
    "FrontRight": "Closed",
    "RearLeft": "Closed",
    "RearRight": "Closed",
    "Trunk": "Closed"
  },
  "CANData": {
    "BrakePressure": 2.3,
    "ThrottlePosition": 25.4,
    "GearPosition": "Drive",
    "TurnSignal": "Off"
  }
}


Model: phi35-vision-instruct
Usage:
	Prompt tokens: 350
	Total tokens: 716
	Completion tokens: 366


### **Proposal Agent**
---
- **Role:** Proposes actions or suggestions based on the recognized environment and user intent.
- **Functions:**
  - Performs intent analysis to understand user needs.
  - Sets target values for actions or goals.
  - Calculates discrepancies and defines resolution strategies.
- **Features:** Includes a user preference processing function to make personalized suggestions.
- **Demo Architecture:**
  - Use the output from Environmental Recognition Agent as input for this Proposal Agent
  - The agent consists of a singble SLM(Phi-3.5).
  - Uge Phi-3.5 model with Serverless API that is provided by [Azure AI Model Catalog](https://learn.microsoft.com/en-us/azure/machine-learning/concept-model-catalog?view=azureml-api-2).

In [11]:
proposal_agent_client = ChatCompletionsClient(
    endpoint=os.environ["AZURE_INFERENCE_ENDPOINT_PHI35"],
    credential=AzureKeyCredential(os.environ["AZURE_INFERENCE_CREDENTIAL_PHI35"]),
)

In [12]:
# Check the status of the model
proposal_agent_model_info = proposal_agent_client.get_model_info()
print("Model name:", proposal_agent_model_info.model_name)
print("Model type:", proposal_agent_model_info.model_type)
print("Model provider name:", proposal_agent_model_info.model_provider_name)

Model name: phi35-mini-instruct
Model type: chat-completion
Model provider name: Phi


In [13]:
available_action_agents = [
	"temperature_control_agent",
	"window_control_agent",
	"entertainment_control_agent",
]

user_preferences = {
	"preferred_temperature": 20,
	"preferred_humidity": 40,
	"preferred_noise_level": "low",
	"preferred_air_quality": "good",
	"preferred_lighting": "dim",
	"preferred_music": "cheerful",
	"favorite_places": ["beach"],
}

In [14]:
output_example = {
    "target_agent": "Temperature Control Agent",
    "ideal_state": {
        "temperature": 26,
        "humidity": 50
    },
    "instructions": {
        "action": "set_temperature",
        "value": 26
    }
}

In [15]:
# Prompt definition
proposal_agent_system_message = f"""
You are an agent tasked with leveraging in-car systems to provide an optimal and comfortable environment for the user.
Based on the current state, user preferences, and available action agents, suggest an ideal comfortable state.
Then, determine the appropriate agent and the instructions needed to achieve the ideal state.
You MUST output ONLY the json as shown in the output example. You MUST NOT includie any other additional comment or text.
"""

proposal_agent_user_message = f"""
# Input

## Current Environment State
{env_agent_response.choices[0].message.content}

## Available Action Agents
{available_action_agents}

## User Preferences
{user_preferences}

# Output example:
{output_example}

"""

In [16]:
from azure.ai.inference.models import SystemMessage, UserMessage

proposal_agent_response = proposal_agent_client.complete(
    messages=[
        SystemMessage(content=proposal_agent_system_message),
        UserMessage(content=proposal_agent_user_message),
    ],
)

In [17]:
print("Response:", proposal_agent_response.choices[0].message.content)
print("Model:", proposal_agent_response.model)
print("Usage:")
print("\tPrompt tokens:", proposal_agent_response.usage.prompt_tokens)
print("\tTotal tokens:", proposal_agent_response.usage.total_tokens)
print("\tCompletion tokens:", proposal_agent_response.usage.completion_tokens)

Response:  {
  "target_agent": "temperature_control_agent",
  "ideal_state": {"temperature": 20, "humidity": 40},
  "instructions": {"action": "set_temperature", "value": 20, "humidity": 40}
}

Model: phi35-mini-instruct
Usage:
	Prompt tokens: 657
	Total tokens: 734
	Completion tokens: 77


### **Processing Agent**
---
- **Role:** Executes tasks or commands based on the proposals.
- **Functions:**
  - Handles external service skills and edge-local skills.
  - Executes predefined "Skills" to perform user-requested tasks.
- **Notes:** Skills can include controlling devices, retrieving information, or performing other actionable commands.
- **Demo Architecture:**
  - Use the output from Proposal Agent as input for this Processing Agent
  - The agent consists of a singble SLM(Phi-3.5).
  - Uge Phi-3.5 model with Serverless API that is provided by [Azure AI Model Catalog](https://learn.microsoft.com/en-us/azure/machine-learning/concept-model-catalog?view=azureml-api-2).

#### Temperature Control Agent

In [18]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

def get_azureml_inference(messages):

    allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

    # Request data goes here
    # The example below assumes JSON formatting which may be updated
    # depending on the format your endpoint expects.
    # More information can be found here:
    # https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
    data = {
        "input_data": messages,
        "params": {
            "temperature": 0.1,
            "max_new_tokens": 2048,
            "do_sample": True,
            "return_full_text": False
        }
    }

    body = str.encode(json.dumps(data))

    url = os.environ["AZURE_INFERENCE_ENDPOINT_PHI35_TEMPERATURE"]
    # Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
    api_key = os.environ["AZURE_INFERENCE_CREDENTIAL_PHI35_TEMPERATURE"]
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")


    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        decoded_result = result.decode('utf-8')
        parsed_result = json.loads(decoded_result)
        print(parsed_result)
        return parsed_result["result"]
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))
        return None


Here is the list of in-vehicle API. This might be generated based on the actual API specification, document, code.

In [24]:
temperature_control_list_available_features = [
    {
        "type": "function",
        "function": {
            "name": "set_cabin_temperature",
            "description": "Set cabin temperature to specified degree.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "temperature": {
                        "type": "number",
                        "description": "Target temperature to be set in the cabin."
                    }
                },
                "required": ["temperature"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "auto_adjust_cabin_environment",
            "description": "Automatically adjust the cabin environment.",
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_fan_speed",
            "description": "Set fan speed to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "fan_speed": {
                        "type": "integer",
                        "enum": [1, 2, 3, 4, 5],
                        "description": "Fan speed level (1: Low, 5: High)."
                    }
                },
                "required": ["fan_speed"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_seat_heater_level",
            "description": "Set seat heater to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "level": {
                        "type": "integer",
                        "enum": [0, 1, 2, 3],
                        "description": "Seat heater level (0: Off, 3: Max)."
                    }
                },
                "required": ["level"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_seat_ventilation_level",
            "description": "Set seat ventilation to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "level": {
                        "type": "integer",
                        "enum": [0, 1, 2, 3],
                        "description": "Seat ventilation level (0: Off, 3: Max)."
                    }
                },
                "required": ["level"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_steering_heater_level",
            "description": "Set steering heater to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "level": {
                        "type": "integer",
                        "enum": [0, 1, 2],
                        "description": "Steering heater level (0: Off, 2: Max)."
                    }
                },
                "required": ["level"],
                "additionalProperties": False
            }
        }
    }
]

In [20]:
# Prompt definition
temperature_control_agent_system_message = f"""
You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\n<tools>\n{temperature_control_list_available_features}\n</tools>\nFor each function call return a json object with function name and arguments within <tool_call> </tool_call> tags with the following schema:\n<tool_call>\n{{'arguments': <args-dict>, 'name': <function-name>}}\n</tool_call>\n
"""

temperature_control_agent_user_message = f"""
# Input

## Proposal Contents from the Proposal Agent
{proposal_agent_response.choices[0].message.content}

## User Preferences
{user_preferences}

"""

In [21]:
messages=[
	{"role": "system", "content": temperature_control_agent_system_message},
	{"role": "user", "content": temperature_control_agent_user_message},
]

In [22]:
response = get_azureml_inference(messages)

{'result': ' <tool_call>\n{"name": "set_cabin_temperature", "arguments": {"temperature": 20}}\n</tool_call>'}


#### Low grade model

In [25]:
temperature_control_list_available_features_low = [
    {
        "type": "function",
        "function": {
            "name": "set_cabin_temperature",
            "description": "Set cabin temperature to specified degree.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "temperature": {
                        "type": "number",
                        "description": "Target temperature to be set in the cabin."
                    }
                },
                "required": ["temperature"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "set_fan_speed",
            "description": "Set fan speed to specified level.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "fan_speed": {
                        "type": "integer",
                        "enum": [1, 2, 3, 4, 5],
                        "description": "Fan speed level (1: Low, 5: High)."
                    }
                },
                "required": ["fan_speed"],
                "additionalProperties": False
            }
        }
    },
]

In [27]:
# Prompt definition for low grade model
temperature_control_agent_system_message_low = f"""
You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\n<tools>\n{temperature_control_list_available_features_low}\n</tools>\nFor each function call return a json object with function name and arguments within <tool_call> </tool_call> tags with the following schema:\n<tool_call>\n{{'arguments': <args-dict>, 'name': <function-name>}}\n</tool_call>\n
"""

In [28]:
messages_low=[
	{"role": "system", "content": temperature_control_agent_system_message_low},
	{"role": "user", "content": temperature_control_agent_user_message},
]

In [30]:
response = get_azureml_inference(messages_low)

{'result': " <tool_call>\n{'arguments': {'temperature': 20}, 'name':'set_cabin_temperature'}\n</tool_call>"}


## Feedback loop

Users will evaluate the actions performed by the AI agent, judging whether they are satisfactory or not. If they are dissatisfied with an action, they will likely take corrective measures themselves (e.g., raising the temperature if the AI lowered it too much).

To consistently reflect user preferences, the system will analyze the history of AI agent actions and corresponding user responses. This data will be used to periodically update the user profile.

The updated profile will then serve as prompot for the AI agent to determine and execute actions it deems more appropriate.

In [31]:
user_preferences = {
	"preferred_temperature": 28, # Updated preferred temperature!
	"preferred_humidity": 40,
	"preferred_noise_level": "low",
	"preferred_air_quality": "good",
	"preferred_lighting": "dim",
	"preferred_music": "cheerful",
	"favorite_places": ["beach"],
}

In [32]:
proposal_agent_user_message = f"""
# Input

## Current Environment State
{env_agent_response.choices[0].message.content}

## Available Action Agents
{available_action_agents}

## User Preferences
{user_preferences}

# Output example:
{output_example}

"""

In [35]:
from azure.ai.inference.models import SystemMessage, UserMessage

proposal_agent_response = proposal_agent_client.complete(
    messages=[
        SystemMessage(content=proposal_agent_system_message),
        UserMessage(content=proposal_agent_user_message),
    ],
)

In [36]:
print("Response:", proposal_agent_response.choices[0].message.content)

Response:  {
  "target_agent": "temperature_control_agent",
  "ideal_state": {
    "temperature": 28,
    "humidity": 40
  },
  "instructions": {
    "action": "set_temperature",
    "value": 28,
    "humidity": 40
  }
}




In [38]:
temperature_control_agent_user_message = f"""
# Input

## Proposal Contents from the Proposal Agent
{proposal_agent_response.choices[0].message.content}

## User Preferences
{user_preferences}

"""

In [39]:
messages=[
	{"role": "system", "content": temperature_control_agent_system_message},
	{"role": "user", "content": temperature_control_agent_user_message},
]

In [40]:
response = get_azureml_inference(messages)

{'result': ' <tool_call>\n{"name": "set_cabin_temperature", "arguments": {"temperature": 28}}\n</tool_call>'}
